In [1]:
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import STOPWORDS,WordCloud
import gensim
from gensim.models import CoherenceModel
from gensim import corpora
import pandas as pd
from pprint import pprint
import string
import os
import re
from extract_dataframe import ExtractTweets
from clean_tweets_dataframe import TweetCleanser

ModuleNotFoundError: No module named 'wordcloud'

In [ ]:
# dataframe from extracted tweets
tweets_df = ExtractTweets("data/Economic_Twitter_Data.json")
tweets_df = extracted_tweets.get_tweet_df(save=False)
tweets_df.dropna()

,Unnamed: 0,created_at,source,original_text,clean_text,sentiment,polarity,subjectivity,lang,favorite_count,retweet_count,original_author,screen_count,followers_count,friends_count,possibly_sensitive,hashtags,user_mentions,place,place_coord_boundaries,timestamp


In [ ]:
len(tweets_df)

5621

In [ ]:
class PrepareData:
  def __init__(self,df):
    self.df=df
    
  def preprocess_data(self):
    tweets_df = self.df.loc[self.df['lang'] =="en"]

    
    #text Preprocessing
    tweets_df['clean_text']=tweets_df['clean_text'].astype(str)
    tweets_df['clean_text'] = tweets_df['clean_text'].apply(lambda x: x.lower())
    tweets_df['clean_text']= tweets_df['clean_text'].apply(lambda x: x.translate(str.maketrans(' ', ' ', string.punctuation)))
    
    #Converting tweets to list of words For feature engineering
    sentence_list = [tweet for tweet in tweets_df['clean_text']]
    word_list = [sent.split() for sent in sentence_list]

    #Create dictionary which contains Id and word 
    word_to_id = corpora.Dictionary(word_list)
    corpus_1= [word_to_id.doc2bow(tweet) for tweet in word_list]



    
    return word_list, word_to_id, corpus_1


In [ ]:
PrepareData_obj=PrepareData(tweets_df)
word_list ,id2word,corpus=PrepareData_obj.preprocess_data()

In [ ]:
print(corpus)

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 2), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1)], [(26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1)], [(29, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 2), (53, 1), (54, 1), (55, 2), (56, 2), (57, 1)], [(58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 2), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1)], [(26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1)], [(29, 1), (75, 1), (76, 1), (77, 1), (78, 1), (79, 1), (80, 1), (81, 1), (82, 1), (83, 1), (84, 1), (85, 1), (86, 1),

In [ ]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus,
                                           id2word=id2word,
                                           num_topics=5, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [ ]:


pprint(lda_model.show_topics(formatted=False))

[(0,
  [('carbon', 0.033523474),
   ('climate', 0.022791425),
   ('gt', 0.021368595),
   ('bp', 0.012794656),
   ('change', 0.01143688),
   ('land', 0.008486307),
   ('latest', 0.007863012),
   ('large', 0.0076355455),
   ('intense', 0.0074943104),
   ('americans', 0.006986857)]),
 (1,
  [('carbon', 0.022609165),
   ('first', 0.011645413),
   ('money', 0.01162081),
   ('emissions', 0.011445911),
   ('the', 0.010748187),
   ('reduce', 0.0100996215),
   ('see', 0.009358184),
   ('s', 0.009275624),
   ('it', 0.008924053),
   ('this', 0.008717043)]),
 (2,
  [('nt', 0.018154405),
   ('meat', 0.015851557),
   ('carbon', 0.015013237),
   ('uk', 0.014227561),
   ('like', 0.012811348),
   ('money', 0.012667962),
   ('giving', 0.012188516),
   ('june', 0.012113796),
   ('rt', 0.011740563),
   ('th', 0.01171801)]),
 (3,
  [('carbon', 0.042917076),
   ('tax', 0.028562853),
   ('protecting', 0.027864862),
   ('rights', 0.027779127),
   ('amp', 0.023900261),
   ('oil', 0.023304556),
   ('emissions',

In [ ]:
# Compute Perplexity

#It's a measure of how good the model is. The lower the better. Perplexity is a negative value
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  
doc_lda = lda_model[corpus]


# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=word_list, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\n Ldamodel Coherence Score/Accuracy on Tweets: ', coherence_lda)


Perplexity:  -6.74028064250165

 Ldamodel Coherence Score/Accuracy on Tweets:  0.5846626375222727


In [ ]:
import pyLDAvis.gensim_models as gensimvis
import pickle 
import pyLDAvis
# Visualize the topics
pyLDAvis.enable_notebook()

LDAvis_prepared = gensimvis.prepare(lda_model, corpus, id2word)
LDAvis_prepared

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.298347  0.187598       1        1  27.315574
1     -0.011718  0.019788       2        1  22.293761
2      0.303998  0.197470       3        1  20.110988
4     -0.029315 -0.162216       4        1  17.137186
0      0.035381 -0.242640       5        1  13.142490, topic_info=            Term        Freq       Total Category  logprob  loglift
973          tax  617.000000  617.000000  Default  30.0000  30.0000
966   protecting  602.000000  602.000000  Default  29.0000  29.0000
971       rights  600.000000  600.000000  Default  28.0000  28.0000
364   government  397.000000  397.000000  Default  27.0000  27.0000
208          oil  504.000000  504.000000  Default  26.0000  26.0000
...          ...         ...         ...      ...      ...      ...
1752       large   79.340788  123.059359   Topic5  -4.8749   1.5904
37          land   88.181035  394.623878   Topic5  -4.7693   0.5308
558       future   72.592915  115.870899   Topic5  -4.9638   1.5617
114         kuda   71.132622  200.909601   Topic5  -4.9841   0.9910
253    footprint   71.458386  317.045182   Topic5  -4.9796   0.5394

[244 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
468       2  0.991148  absolutely
58        1  0.124406     account
58        2  0.163283     account
58        3  0.711449     account
1186      3  0.995155        accs
...     ...       ...         ...
300       3  0.912999       would
1208      3  0.995155        xtra
277       2  0.849107        year
277       3  0.144348        year
373       4  0.996612        zero

[265 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 2, 3, 5, 1])